## Часть 1
Напишите скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227). Затем нужно перейти на сайты региональных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название региона
- название ТИК
- номер УИК
- 20 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

### Часть 2
Используя скаченные данные и `Pandas` (или `PySpark`) посчитайте:
- явку (%) по всем регионам, результат отсортировать по убыванию
- выберите произвольного кандидата и найдите тот избиратльный участок, на котором он получил наибольший результат  (%, учитывать участки на которых проголосовало больше 300 человек)
- найдите регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна
- посчитайте дисперсию по явке для каждого региона (учитывать УИК)
- для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/election_data.csv', sep='\t', encoding='utf-8')
regions = df.Регион.unique()

In [3]:
regions_appearance = {}
for region in regions:
    total = df.loc[df.Регион == region, df.columns[5]].sum()
    counted = df.loc[df.Регион == region, df.columns[7]].sum()
    appearance = counted / total * 100
    regions_appearance[region] = appearance
sorted(regions_appearance.items(), key=lambda t: -t[1])

[(' Чеченская Республика', 92.37943323561534),
 (' Кабардино-Балкарская Республика', 87.519626906313),
 (' Республика Дагестан', 86.18083655713164),
 (' Республика Северная Осетия - Алания', 85.73365153263825),
 (' Республика Ингушетия', 84.56886972860325),
 (' Карачаево-Черкесская Республика', 83.74121327086533),
 (' город Севастополь', 81.1228277082105),
 (' Ямало-Ненецкий автономный округ', 79.43875261640483),
 (' Кемеровская область', 78.9394299250778),
 (' Республика Тыва', 78.77189303133453),
 (' Тюменская область', 78.3086092265614),
 (' Республика Крым', 78.23055579528453),
 (' Республика Татарстан (Татарстан)', 76.91193222663378),
 (' Республика Башкортостан', 76.2702842604394),
 (' Республика Мордовия', 75.00784190715181),
 (' Чукотский автономный округ', 74.90556690816769),
 (' Брянская область', 72.83976086224317),
 (' Республика Бурятия', 72.4575722243814),
 (' Краснодарский край', 71.6545580206834),
 (' Республика Адыгея (Адыгея)', 71.51332964164227),
 (' Чувашская Респуб

In [4]:
df.loc[df[df.columns[20]]>300,[df.columns[20],df.columns[3]]].max()

Собчак Ксения Анатольевна   NaN
Номер УИК                   NaN
dtype: float64

In [5]:
differences = {}
for region in regions:
    r = df[df.Регион == region]
    for district in r[r.columns[2]].unique():
        max_appearance = r.loc[df[df.columns[2]] == district, df.columns[7]].max()
        min_appearance = r.loc[df[df.columns[2]] == district, df.columns[7]].min()
        differences[region] = max_appearance - min_appearance
max(differences, key=differences.get)

' город Москва'

In [6]:
dis = {}
for region in regions:
    d = df.loc[df.Регион == region, df.columns[7]].var()
    dis[region] = d
dis

{' Республика Адыгея (Адыгея)': 298599.2310173984,
 ' Республика Алтай': 139849.64989197906,
 ' Республика Башкортостан': 279570.2029596715,
 ' Республика Бурятия': 194877.62826567033,
 ' Республика Дагестан': 416918.8062860099,
 ' Республика Ингушетия': 349489.99474023195,
 ' Кабардино-Балкарская Республика': 371203.2854227682,
 ' Республика Калмыкия': 129633.63560159356,
 ' Карачаево-Черкесская Республика': 271935.5124143427,
 ' Республика Карелия': 208307.7855509686,
 ' Республика Коми': 212074.0863475421,
 ' Республика Крым': 250377.7600111477,
 ' Республика Марий Эл': 180505.29964332853,
 ' Республика Мордовия': 220159.5156148449,
 ' Республика Саха (Якутия)': 208031.99857589818,
 ' Республика Северная Осетия - Алания': 430259.3378184506,
 ' Республика Татарстан (Татарстан)': 330638.1387166897,
 ' Республика Тыва': 366036.62373145967,
 ' Удмуртская Республика': 190925.99049096528,
 ' Республика Хакасия': 177366.56605679053,
 ' Чеченская Республика': 597639.6298023537,
 ' Чувашская